In [ ]:
import face_recognition
import cv2
import os
# from google.colab import drive
# from google.colab.patches import cv2_imshow

# # Mount Google Drive
# drive.mount('/content/drive')

KNOWN_IMAGE_DIR = "./test"
VIDEO_PATH = "./test/fan__official_trailer__shah_rukh_khan.mp3"
OUTPUT_VIDEO_PATH = "./test/fan__official_trailer__shah_rukh_khan_2.mp3"

def load_known_faces(known_image_dir):
    known_face_encodings = []
    known_face_names = []

    # Load all known faces from the directory
    for filename in os.listdir(known_image_dir):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(known_image_dir, filename)
            name = os.path.splitext(filename)[0]  # Extract name from filename
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                known_face_encodings.append(face_encodings[0])
                known_face_names.append(name)
                print(f"Loaded {name} from {filename}")

    return known_face_encodings, known_face_names

def recognize_faces_in_video(video_path, known_face_encodings, known_face_names, output_video_path):
    # Open the video file
    video_capture = cv2.VideoCapture(video_path)

    # Get video properties
    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))

    # Define codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v') # or 'XVID' for AVI
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop over each face found in the frame
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for any of the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with the name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        # Write the frame into the output video
        out.write(frame)

    # Release everything when the job is finished
    video_capture.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces from the specified directory
    known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)

    # Process video and save output
    recognize_faces_in_video(VIDEO_PATH, known_face_encodings, known_face_names, OUTPUT_VIDEO_PATH)

    # Display the result
    print("Video processing complete. Output video saved to Google Drive.")


## Working Code 1

In [4]:
import face_recognition
import cv2
import dlib
import os

KNOWN_IMAGE_DIR = "./test"
OUTPUT_VIDEO_PATH = "./test/Shahrukh_Khan_Hairstyle.jpeg"

def load_known_faces(known_image_dir):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(known_image_dir):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(known_image_dir, filename)
            name = os.path.splitext(filename)[0]  
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                known_face_encodings.append(face_encodings[0])
                known_face_names.append(name)
                print(f"Loaded {name} from {filename}")

    return known_face_encodings, known_face_names

def recognize_faces_in_webcam(known_face_encodings, known_face_names, output_video_path):
    video_capture = cv2.VideoCapture(0)  

    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 20.0  # Adjust FPS if needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' for AVI
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop over each face found in the frame
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for any of the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_face_names[best_match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with the name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        out.write(frame)
        cv2.imshow('Webcam Feed', frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces from the specified directory
    known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)

    # Process webcam feed and save output
    recognize_faces_in_webcam(known_face_encodings, known_face_names, OUTPUT_VIDEO_PATH)

    # Display the result
    print("Webcam processing complete. Output video saved to folder")

Loaded Ibtasam from Ibtasam.jpg


KeyboardInterrupt: 

: 

## Code for Processing a Video File

In [3]:
import face_recognition
import cv2
import os
from datetime import datetime
from PIL import UnidentifiedImageError, Image

KNOWN_IMAGE_DIR = "./test"
UNKNOWN_IMAGE_DIR = "./unknown_images"
VIDEO_PATH = "fan__official_trailer__shah_rukh_khan.mp3"  # Replace with your video file path

def load_known_faces(known_image_dir):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(known_image_dir):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(known_image_dir, filename)
            name = os.path.splitext(filename)[0]

            try:
                image = face_recognition.load_image_file(image_path)
                face_encodings = face_recognition.face_encodings(image)

                if face_encodings:
                    known_face_encodings.append(face_encodings[0])
                    known_face_names.append(name)
                else:
                    print(f"No face found in {filename}. Skipping.")

            except UnidentifiedImageError:
                print(f"Cannot identify image file {filename}. Skipping.")

    return known_face_encodings, known_face_names

def recognize_faces_in_video(known_face_encodings, known_face_names, video_path, unknown_image_dir):
    video_capture = cv2.VideoCapture(video_path)

    if not os.path.exists(unknown_image_dir):
        os.makedirs(unknown_image_dir)

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True not in matches:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                unknown_image_path = os.path.join(unknown_image_dir, f"unknown_{timestamp}.jpg")
                cv2.imwrite(unknown_image_path, frame)
            else:
                best_match_index = matches.index(True)
                name = known_face_names[best_match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        cv2.imshow('Video Feed', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)
    recognize_faces_in_video(known_face_encodings, known_face_names, VIDEO_PATH, UNKNOWN_IMAGE_DIR)


Cannot identify image file Shahrukh.jpg. Skipping.


## Code for Real-Time Webcam Processing

In [2]:
import face_recognition
import cv2
import os
from datetime import datetime
from PIL import UnidentifiedImageError, Image

KNOWN_IMAGE_DIR = "./test"
UNKNOWN_IMAGE_DIR = "./unknown_images"

def load_known_faces(known_image_dir):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(known_image_dir):
        if filename.endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(known_image_dir, filename)
            name = os.path.splitext(filename)[0]

            try:
                image = face_recognition.load_image_file(image_path)
                face_encodings = face_recognition.face_encodings(image)

                if face_encodings:
                    known_face_encodings.append(face_encodings[0])
                    known_face_names.append(name)
                else:
                    print(f"No face found in {filename}. Skipping.")

            except UnidentifiedImageError:
                print(f"Cannot identify image file {filename}. Skipping.")

    return known_face_encodings, known_face_names

def recognize_faces_in_webcam(known_face_encodings, known_face_names, unknown_image_dir):
    video_capture = cv2.VideoCapture(0)

    if not os.path.exists(unknown_image_dir):
        os.makedirs(unknown_image_dir)

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            if True not in matches:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                unknown_image_path = os.path.join(unknown_image_dir, f"unknown_{timestamp}.jpg")
                cv2.imwrite(unknown_image_path, frame)
            else:
                best_match_index = matches.index(True)
                name = known_face_names[best_match_index]

            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        cv2.imshow('Webcam Feed', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)
    recognize_faces_in_webcam(known_face_encodings, known_face_names, UNKNOWN_IMAGE_DIR)


UnidentifiedImageError: cannot identify image file './test\\Shahrukh.jpg'

In [4]:
import face_recognition
import cv2
import dlib
import os
from datetime import datetime

KNOWN_IMAGE_DIR = "./test"
OUTPUT_VIDEO_PATH = "./test/Shahrukh_Khan_Hairstyle.jpeg"
UNKNOWN_IMAGE_DIR = "./picture"  # Directory to save images of unknown faces

def load_known_faces(known_image_dir):
    known_face_encodings = []
    known_face_names = []

    for filename in os.listdir(known_image_dir):
        if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
            image_path = os.path.join(known_image_dir, filename)
            name = os.path.splitext(filename)[0]  
            image = face_recognition.load_image_file(image_path)
            face_encodings = face_recognition.face_encodings(image)

            if face_encodings:
                known_face_encodings.append(face_encodings[0])
                known_face_names.append(name)
                print(f"Loaded {name} from {filename}")

    return known_face_encodings, known_face_names

def save_unknown_face(frame, face_location):
    # Ensure the unknown image directory exists
    os.makedirs(UNKNOWN_IMAGE_DIR, exist_ok=True)

    # Extract the face region
    top, right, bottom, left = face_location
    face_image = frame[top:bottom, left:right]

    # Create a unique filename with a timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"unknown_{timestamp}.jpg"
    filepath = os.path.join(UNKNOWN_IMAGE_DIR, filename)

    # Save the face image
    cv2.imwrite(filepath, face_image)
    print(f"Saved unknown face to {filepath}")

def recognize_faces_in_webcam(known_face_encodings, known_face_names, output_video_path):
    video_capture = cv2.VideoCapture(0)  

    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 20.0  # Adjust FPS if needed
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' for AVI
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Loop over each face found in the frame
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            # See if the face is a match for any of the known faces
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"

            # Use the known face with the smallest distance to the new face
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = face_distances.argmin()
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
            else:
                save_unknown_face(frame, (top, right, bottom, left))  # Save screenshot of unknown face

            # Draw a rectangle around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with the name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

        out.write(frame)
        cv2.imshow('Webcam Feed', frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    # Load known faces from the specified directory
    known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)

    # Process webcam feed and save output
    recognize_faces_in_webcam(known_face_encodings, known_face_names, OUTPUT_VIDEO_PATH)

    # Display the result
    print("Webcam processing complete. Output video saved to folder")


Loaded Ibtasam from Ibtasam.jpg
Saved unknown face to ./picture\unknown_20240831_163143.jpg
Saved unknown face to ./picture\unknown_20240831_163149.jpg
Saved unknown face to ./picture\unknown_20240831_163151.jpg
Saved unknown face to ./picture\unknown_20240831_163153.jpg
Saved unknown face to ./picture\unknown_20240831_163154.jpg
Saved unknown face to ./picture\unknown_20240831_163156.jpg
Saved unknown face to ./picture\unknown_20240831_163158.jpg


KeyboardInterrupt: 

: 

In [1]:
import face_recognition
import cv2
import dlib
import os
from datetime import datetime

KNOWN_IMAGE_DIR = "./test"
UNKNOWN_IMAGE_DIR = "./picture"  # Directory to save images of unknown faces

def load_known_faces(known_image_dir):
  known_face_encodings = []
  known_face_names = []

  for filename in os.listdir(known_image_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
      image_path = os.path.join(known_image_dir, filename)
      name = os.path.splitext(filename)[0]
      image = face_recognition.load_image_file(image_path)
      face_encodings = face_recognition.face_encodings(image)

      if face_encodings:
        known_face_encodings.append(face_encodings[0])
        known_face_names.append(name)
        print(f"Loaded {name} from {filename}")

  return known_face_encodings, known_face_names

def save_unknown_face(frame, face_location):
  # Ensure the unknown image directory exists
  os.makedirs(UNKNOWN_IMAGE_DIR, exist_ok=True)

  # Extract the face region
  top, right, bottom, left = face_location
  face_image = frame[top:bottom, left:right]

  # Create a unique filename with a timestamp
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  filename = f"unknown_{timestamp}.jpg"
  filepath = os.path.join(UNKNOWN_IMAGE_DIR, filename)

  # Save the face image
  cv2.imwrite(filepath, face_image)
  print(f"Saved unknown face to {filepath}")

def recognize_faces_in_webcam(known_face_encodings, known_face_names):
  video_capture = cv2.VideoCapture(0)  

  frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
  frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
  fps = 20.0  # Adjust FPS if needed
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'XVID' for AVI

  while True:
    ret, frame = video_capture.read()
    if not ret:
      print("Failed to grab frame")
      break

    # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Find all the faces and face encodings in the frame
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Loop over each face found in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
      # See if the face is a match for any of the known faces
      matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
      name = "Unknown"

      # Use the known face with the smallest distance to the new face
      face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
      best_match_index = face_distances.argmin()
      if matches[best_match_index]:
        name = known_face_names[best_match_index]
      else:
        save_unknown_face(frame, (top, right, bottom, left))  # Save screenshot of unknown face

      # Draw a rectangle around the face
      cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

      # Draw a label with the name below the face
      cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
      font = cv2.FONT_HERSHEY_DUPLEX
      cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    cv2.imshow('Webcam Feed', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break

  video_capture.release()
  cv2.destroyAllWindows()

if __name__ == "__main__":
  # Load known faces from the specified directory
  known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)

  # Process webcam feed and save output
  recognize_faces_in_webcam(known_face_encodings, known_face_names)

  # Display the result
  print("Webcam processing complete. Output video saved to folder")

Loaded Ibtasam from Ibtasam.jpg
Saved unknown face to ./picture\unknown_20240831_173930.jpg
Saved unknown face to ./picture\unknown_20240831_173940.jpg
Saved unknown face to ./picture\unknown_20240831_173942.jpg
Saved unknown face to ./picture\unknown_20240831_173942.jpg
Saved unknown face to ./picture\unknown_20240831_173944.jpg
Saved unknown face to ./picture\unknown_20240831_173946.jpg
Saved unknown face to ./picture\unknown_20240831_173947.jpg
Saved unknown face to ./picture\unknown_20240831_173948.jpg
Saved unknown face to ./picture\unknown_20240831_173953.jpg
Saved unknown face to ./picture\unknown_20240831_173956.jpg
Saved unknown face to ./picture\unknown_20240831_173957.jpg
Saved unknown face to ./picture\unknown_20240831_174000.jpg
Saved unknown face to ./picture\unknown_20240831_174000.jpg
Saved unknown face to ./picture\unknown_20240831_174002.jpg
Saved unknown face to ./picture\unknown_20240831_174003.jpg
Saved unknown face to ./picture\unknown_20240831_174006.jpg
Saved un

KeyboardInterrupt: 

: 

In [1]:
import face_recognition
import cv2
import dlib
import os
from datetime import datetime

KNOWN_IMAGE_DIR = "./test"
UNKNOWN_IMAGE_DIR = "./picture"  # Directory to save images of unknown faces

def load_known_faces(known_image_dir):
  known_face_encodings = []
  known_face_names = []

  for filename in os.listdir(known_image_dir):
    if filename.endswith(".jpg") or filename.endswith(".jpeg") or filename.endswith(".png"):
      image_path = os.path.join(known_image_dir, filename)
      name = os.path.splitext(filename)[0]
      image = face_recognition.load_image_file(image_path)
      face_encodings = face_recognition.face_encodings(image)

      if face_encodings:
        known_face_encodings.append(face_encodings[0])
        known_face_names.append(name)
        print(f"Loaded {name} from {filename}")

  return known_face_encodings, known_face_names

def save_unknown_face(frame, face_location):
  # Ensure the unknown image directory exists
  os.makedirs(UNKNOWN_IMAGE_DIR, exist_ok=True)

  # Extract the face region
  top, right, bottom, left = face_location
  face_image = frame[top:bottom, left:right]

  # Create a unique filename with a timestamp
  timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
  filename = f"unknown_{timestamp}.jpg"
  filepath = os.path.join(UNKNOWN_IMAGE_DIR, filename)

  # Save the face image
  cv2.imwrite(filepath, face_image)
  print(f"Saved unknown face to {filepath}")

def recognize_faces_in_webcam(known_face_encodings, known_face_names):
    video_capture = cv2.VideoCapture(0)  

    while True:
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Convert the image from BGR (OpenCV format) to RGB (face_recognition format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find all the faces and face encodings in the frame
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        # Check the number of faces detected
        if len(face_locations) != 1:
            print("Please be one person in front of the camera")
            cv2.imshow('Webcam Feed', frame)
        else:
            # Loop over each face found in the frame
            for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
                # See if the face is a match for any of the known faces
                matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
                name = "Unknown"

                # Use the known face with the smallest distance to the new face
                face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
                best_match_index = face_distances.argmin()
                if matches[best_match_index]:
                    name = known_face_names[best_match_index]
                else:
                    save_unknown_face(frame, (top, right, bottom, left))  # Save screenshot of unknown face

                # Draw a rectangle around the face
                cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

                # Draw a label with the name below the face
                cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
                font = cv2.FONT_HERSHEY_DUPLEX
                cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

            cv2.imshow('Webcam Feed', frame)

        # Break loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
  # Load known faces from the specified directory
  known_face_encodings, known_face_names = load_known_faces(KNOWN_IMAGE_DIR)

  # Process webcam feed and save output
  recognize_faces_in_webcam(known_face_encodings, known_face_names)

  # Display the result
  print("Webcam processing complete. Output video saved to folder")

Loaded Ibtasam from Ibtasam.jpg
Saved unknown face to ./picture\unknown_20240831_192819.jpg
Please be one person in front of the camera
Please be one person in front of the camera
Saved unknown face to ./picture\unknown_20240831_192822.jpg
Saved unknown face to ./picture\unknown_20240831_192823.jpg
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Saved unknown face to ./picture\unknown_20240831_192837.jpg
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in front of the camera
Please be one person in 

KeyboardInterrupt: 

: 